In [1]:
import numpy as np
import h5py
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation

/Users/james/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

In [3]:
x_train= {}
x_test = {}
y_train = {}
y_test= {}
location_train= {}
location_test= {}
feature_all_train = {}
feature_all_test= {}
aoa_train={}
aoa_test={}
toa_train={}
toa_test={}
rss_train={}
rss_test={}
f = h5py.File('../data_cross_250')
for i in range(4):
    x_train[i] = f[str(i)+'/loc_x_train'][:]
    x_test[i] = f[str(i)+'/loc_x_test'][:]
    y_train[i] = f[str(i)+'/loc_y_train'][:] 
    y_test[i] = f[str(i)+'/loc_y_test'][:]
    location_train[i] = f[str(i)+'/loc_train'][:,:]
    location_test[i] = f[str(i)+'/loc_test'][:,:]
    feature_all_train[i] =f[str(i)+'/feature_train'][:,:]
    feature_all_test[i] = f[str(i)+'/feature_test'][:,:]
    aoa_train[i] =f[str(i)+'/aoa_train'][:,:]
    aoa_test[i] = f[str(i)+'/aoa_test'][:,:]
    toa_train[i] =f[str(i)+'/toa_train'][:,:]
    toa_test[i] = f[str(i)+'/toa_test'][:,:]
    rss_train[i] = f[str(i)+'/toa_train'][:,:]
    rss_test[i] = f[str(i)+'/toa_test'][:,:]
    
aoa_toa_train={}
aoa_toa_test ={}
aoa_toa_rss_train={}
aoa_toa_rss_test ={}
for i in range(4):
    aoa_toa_train[i] = feature_all_train[i][:,6:18]
    aoa_toa_test[i] = feature_all_test[i][:,6:18]
    aoa_toa_rss_train[i] = feature_all_train[i][:,:18]
    aoa_toa_rss_test[i] = feature_all_test[i][:,:18]

In [4]:
def data_generator(X, y):
    for i in range(20):
        a= np.arange(200)
        np.random.shuffle(a)
        indices_batch = a[i*10:i*10+10]
        X_batch = np.take(X, indices_batch, axis=0)
        y_batch = np.take(y, indices_batch, axis=0)
        yield X_batch, y_batch

In [14]:
average_dis_error = {}
regpre_x ={}
regpre_y = {}
r2_x = {}
r2_y = {}
pcc_x = {}
pcc_y= {}
dis_error = {}

model = Sequential()    # Feedforward
model.add(Dense(10, input_dim=12, kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dense(6, kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dense(2, kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dense(1))
rms = keras.optimizers.Adam(lr=0.0001,epsilon=1e-6)
model.compile(loss='mean_squared_error', optimizer=rms)

for i in range(4):
    reg_x = model.fit_generator(data_generator(aoa_toa_train[i], y= x_train[i]),
                    steps_per_epoch=1,epochs=1000)
    regpre_x[i] = model.predict(aoa_toa_test[i])
    reg_y = model.fit_generator(data_generator(aoa_toa_train[i], y= y_train[i]),
                    steps_per_epoch=1,epochs=1000)
    regpre_y[i] = model.predict(aoa_toa_test[i])
 
    r2_x[i] = r2_score(regpre_x[i],x_test[i])
    pcc_x[i] = pearsonr(regpre_x[i],x_test[i])
    r2_y[i] = r2_score(regpre_y[i],y_test[i])
    pcc_y[i] = pearsonr(regpre_y[i],y_test[i])
    

    dis_x = {}
    dis_y = {}
    dis = {}
    for j in range(50):
        dis_y[j] = regpre_y[i][j] - y_test[i][j]
        dis_x[j] = regpre_x[i][j] - x_test[i][j]
        dis[j] = (dis_x[j]**2 + dis_y[j]**2)**(0.5)
    sum = np.array([])
    for j in range(50):
        sum = np.concatenate((sum,dis[j]))
    dis_error[i] = np.mean(abs(sum))
sum = 0 
for i in range(4):
    sum = sum+dis_error[i]
average_dis_error = sum/4.0

Epoch 1/1000
1/1 [==============================] - 0s 236ms/step - loss: 35233808.0000
Epoch 2/1000
1/1 [==============================] - 0s 4ms/step - loss: 51735268.0000
Epoch 3/1000
1/1 [==============================] - 0s 4ms/step - loss: 67045568.0000
Epoch 4/1000
1/1 [==============================] - 0s 3ms/step - loss: 54949464.0000
Epoch 5/1000
1/1 [==============================] - 0s 5ms/step - loss: 29960052.0000
Epoch 6/1000
1/1 [==============================] - 0s 4ms/step - loss: 35607960.0000
Epoch 7/1000
1/1 [==============================] - 0s 4ms/step - loss: 51992308.0000
Epoch 8/1000
1/1 [==============================] - 0s 3ms/step - loss: 48646152.0000
Epoch 9/1000
1/1 [==============================] - 0s 2ms/step - loss: 55229908.0000
Epoch 10/1000
1/1 [==============================] - 0s 3ms/step - loss: 37854916.0000
Epoch 11/1000
1/1 [==============================] - 0s 2ms/step - loss: 53976236.0000
Epoch 12/1000
1/1 [==============================]

StopIteration: 